In [ ]:
! wget https://github.com/erkansirin78/datasets/raw/master/personal-cars-classifieds.zip

In [1]:
from my_helpers import MyHelpers
import findspark
findspark.init("/opt/spark")


my_helper_obj = MyHelpers()

## get spark session
session_params_dict = {
    "appName": "Clean Transactions",
    "master": "local[2]",
}

spark = my_helper_obj.get_spark_session(session_params_dict)

# read data
read_params_dict = {
    "format": "csv",
    "header": True,
    "sep": ",",
    "inferSchema": True,
    "path": "all_anonymized_2015_11_2017_03.csv"
}


df = my_helper_obj.get_data(spark_session=spark, read_params=read_params_dict)
df.show(n=5, truncate=False)
df.printSchema()


## format dates
df1 = my_helper_obj.format_dates(date_cols=["date_created", "date_last_seen"], input_df=df)
print("format dates")
df1.show(n=5, truncate=False)
df1.printSchema()


## make_nulls_to_unknown
df2 = my_helper_obj.make_nulls_to_unknown(input_df=df1)
print("make_nulls_to_unknown")
df2.show(5)
df2.printSchema()


## trim_str_cols
df3 = my_helper_obj.trim_str_cols(input_df=df2)
print("trim_str_cols")
df3.show(5)
df3.printSchema()


## write_postgresql
print("write_postgresql")
my_helper_obj.write_to_postgresql(input_df=df3)

# ## Check postgresql
df4 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgresql:5432/mydb") \
    .option("dbtable", "clean3") \
    .option("user", "myuser") \
    .option("password", "mypassword") \
    .option("driver", "org.postgresql.Driver") \
    .load()

print("check_postgresql")
df4.show(5)
spark.stop()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b927e8d4-a88b-406a-ae0d-febd480bc6ea;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 538ms :: artifacts dl 53ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   0 

+-----+-------+-------+----------------+-------------------+------------+---------+----------+--------+------------+----------+----------+---------+--------------------------+-------------------------+---------+
|maker|model  |mileage|manufacture_year|engine_displacement|engine_power|body_type|color_slug|stk_year|transmission|door_count|seat_count|fuel_type|date_created              |date_last_seen           |price_eur|
+-----+-------+-------+----------------+-------------------+------------+---------+----------+--------+------------+----------+----------+---------+--------------------------+-------------------------+---------+
|ford |galaxy |151000 |2011            |2000               |103         |null     |null      |None    |man         |5         |7         |diesel   |2015-11-14 18:10:06.838319|2016-01-27 20:40:15.46361|10584.75 |
|skoda|octavia|143476 |2012            |2000               |81          |null     |null      |None    |man         |5         |5         |diesel   |2015

check_postgresql
+-----+-----+-------+----------------+-------------------+------------+---------+----------+--------+------------+----------+----------+---------+------------+--------------+---------+
|maker|model|mileage|manufacture_year|engine_displacement|engine_power|body_type|color_slug|stk_year|transmission|door_count|seat_count|fuel_type|date_created|date_last_seen|price_eur|
+-----+-----+-------+----------------+-------------------+------------+---------+----------+--------+------------+----------+----------+---------+------------+--------------+---------+
+-----+-----+-------+----------------+-------------------+------------+---------+----------+--------+------------+----------+----------+---------+------------+--------------+---------+

